# Calibration of multiple parameters for ASPICS model, using ABC method

This jupyter notebook is based on the previous efforts from DyME and Prof Nick Malleson (University of Leeds)

- [RAMP-UA Initiative](https://github.com/Urban-Analytics/RAMP-UA/blob/d5973dff007645f1700cded93aaf72298ef84c61/experiments/calibration/abc-1.ipynb)

- [Calibrating Agent-Based Models Using Uncertainty Quantification Methods](https://github.com/Urban-Analytics/uncertainty/blob/master/hm_abc_simple_example.ipyn)

As SPC (Synthetic Population Catalyst) is a tool that helps urban modelling researchers to get synthetic population datasets at national level (currently limitated to England). This tool opens up new challenges/possibilities where external models (multi-level) like Agent-based models -ABM now can be tested in multi regions. However in models with location parameters striclty dependend on the population interactions, internal validation and calibrations process are seen as a relevant and requiered to properly tune this national behaivor. 

### ToDO to make progress in this experiment
- [X] Read the Synt Pop file - Translate to snaphot then ASPICS can read the new dataset.
- [] Read and plot the attributes we need, we could plot
- [] Read the baseline use as priors - Areas to test Leeds ( ideally West Yorkshire), Liverpool, Devon, Manchester (Grand Manchester)

## Background Concepts

- Uncertanity of ABM
- Methods for Calibration
- ABC

In [1]:
import math
import pandas as pd
import sys, os
import matplotlib.pyplot as plt
import numpy as np
import random
from yaml import load, SafeLoader
sys.path.append('../')
from run_aspics import OpenCLRunner
from aspics.loader import setup_sim, create_params
from headless import run_headless
import synthpop_pb2
import convert_snapshot

The following function is based on [SPC scripts](https://github.com/alan-turing-institute/uatk-spc/blob/main/python/protobuf_to_csv.py) the idea is to read the .pb file created with the tool. However, we need to make a translation from the proto file to snapshot which will integarte the data in the way ASPICS need it.

## Read the baseline data. Defined as prior to calibrate the model to a given area
Real observations (number of cases, deaths or hospital admission in the given area)
They need to be made cumulative as this is how they will be compared to the model.

#### Rutland area as test run due it size
The data for no of cases and the gam_cases data were created using [Ramp-UA - Observation Data](https://github.com/Urban-Analytics/RAMP-UA/tree/master/experiments/calibration/observation_data)

In [2]:
# New per day:
gam_cases = pd.read_csv(os.path.join("baseline_data", "gam_rutland_cases.csv"), header=0, names=["Day", "Cases"], )
# Cumulative
OBSERVATIONS = pd.DataFrame( {"Day": gam_cases['Day'], "Cases": gam_cases.cumsum()['Cases']} )
assert OBSERVATIONS.tail(1)['Cases'].values[0] == sum(gam_cases['Cases'])
print(f"Total cases: {sum(gam_cases['Cases'])}")

Total cases: 697


## Run ASPIC using the default parameters

The following cells provide a set of plots to define how the model run with the default parameters ( manually calibrated for Devon area). In this example we use Rutland.

Before everything, we need to translate the .pb (protobufer) file to the snapshot required by ASPCIS [Usage guide](docs/usage_guide.md). You need to do this one time. Once you created your synthetic population file, then run 'run ../convert_snapshot.py -i SPC_data/{YOUR_NEW_AREA}.pb -o ../data/snapshots/{YOUR_NEW_AREA}/cache.npz'


Great now we have the cache.npz file in `data/snapshots`


In [3]:
os.getcwd()

'/Users/fbenitez/PycharmProjects/uatk-aspics/calibration'

In [4]:
os.chdir("../") #Now we need to update the main directory then we can use aspics as the way was created.

In [5]:
PARAMETERS_FILE = 'config/Rutland.yml' # Reading the parameters file
with open(PARAMETERS_FILE, "r") as f:
    parameters = load(f, Loader=SafeLoader)

simulator, snapshot, study_area, iterations = setup_sim(parameters)  #Initial configuration, based on the parameters to run the model
summary, final_state = run_headless(simulator, snapshot, iterations, quiet=False, store_detailed_counts=True) #run the model in a headless mode

Running a manually added parameters simulation based on {'microsim': {'study-area': 'Rutland', 'iterations': 80, 'use-lockdown': False, 'start-date': 10, 'output': True, 'output-every-iteration': False, 'opencl-model': True, 'repetitions': 1}, 'microsim_calibration': {'hazard_individual_multipliers': {'presymptomatic': 1, 'asymptomatic': 0.75, 'symptomatic': 1.0}, 'hazard_location_multipliers': {'Retail': 1.0, 'PrimarySchool': 1.0, 'SecondarySchool': 1.0, 'Home': 1.0, 'Work': 1.0}, 'risk_multiplier': 1.0}, 'disease': {'current_risk_beta': 0.003, 'risk_cap': 5, 'seed_days': 10, 'exposed_dist': 'weibull', 'exposed_mean': 2.56, 'exposed_sd': 0.72, 'presymp_dist': 'weibull', 'presymp_mean': 2.3, 'presymp_sd': 0.35, 'infection_dist': 'lognormal', 'infection_mean': 7, 'infection_sd': 1.3, 'output_switch': True, 'rank_assign': False, 'local_outbreak_timestep': 7, 'local_outbreak': False, 'msoa_infect': 'E02004161', 'number_people_local': 100, 'local_prob_increase': 1.0, 'overweight_sympt_mpli

/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "
Running simulation: 100%|██████████| 80/80 [00:00<00:00, 102.08it/s]


Day 0
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 14
	Symptomatic: 8
	Recovered: 3
	Dead: 0

Day 1
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 13
	Symptomatic: 5
	Recovered: 7
	Dead: 0

Day 2
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 10
	Symptomatic: 5
	Recovered: 10
	Dead: 0

Day 3
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 8
	Symptomatic: 4
	Recovered: 13
	Dead: 0

Day 4
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 4
	Symptomatic: 2
	Recovered: 19
	Dead: 0

Day 5
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 2
	Recovered: 20
	Dead: 0

Day 6
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 1
	Recovered: 21
	Dead: 0

Day 7
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 1
	Recovered: 21
	Dead: 0

Day 8
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 1
	Asymptomatic: 0
	Sym

## Run ASPIC using the manually defined parameters


In [6]:
PARAMETERS_FILE = 'config/Rutland.yml' # Reading the parameters file
with open(PARAMETERS_FILE, "r") as f:
    parameters = load(f, Loader=SafeLoader)

In [7]:
calibration_params = parameters["microsim_calibration"]
disease_params = parameters["disease"]

In [8]:
params_manual = OpenCLRunner.create_params_manually( parameters_file=PARAMETERS_FILE)

Creating parameters manually based on values from Notebook


In [9]:
#sim_params
STUDY_AREA = "Rutland"
USE_LOCKDOWN= False
ITERATIONS = 100  # Number of iterations to run for ( Initially suggestes as 100)
REPETITIONS = 5 #Initially suggested as 5
#OBSERVATIONS IS DECLARED IN THE PREVIOUS CELL
USE_GPU = False
USE_HEALTHIER_POP = False
STORE_DETAILED_COUNTS = False

assert ITERATIONS < len(OBSERVATIONS), \
    f"Have more iterations ({ITERATIONS}) than observations ({len(OBSERVATIONS)})."

In [ ]:
OpenCLRunner.init(
    iterations = ITERATIONS,
    repetitions = REPETITIONS,
    study_area= STUDY_AREA,
    observations = OBSERVATIONS,
    use_gpu = USE_GPU,
    store_detailed_counts = STORE_DETAILED_COUNTS,
    parameters_file = PARAMETERS_FILE,
    use_healthier_pop = False
)

In [ ]:
OpenCLRunner.update(repetitions=10)  # Temporarily use more repetitions to give a good baseline, initially suggested as 10
OpenCLRunner.update(store_detailed_counts=True)

In [10]:
summary, final_state = OpenCLRunner.run_aspics(42, ITERATIONS, STUDY_AREA,params_manual,False,False,USE_GPU,False,10,calibration_params,disease_params,STORE_DETAILED_COUNTS,False)

/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Running a simulation  {42}  based on the study area
Loading snapshot from data/snapshots/Rutland/cache.npz
Snapshot is 7 MB


Running simulation: 100%|██████████| 100/100 [00:00<00:00, 299.93it/s]


Day 0
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 14
	Symptomatic: 8
	Recovered: 3
	Dead: 0

Day 1
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 13
	Symptomatic: 5
	Recovered: 7
	Dead: 0

Day 2
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 10
	Symptomatic: 5
	Recovered: 10
	Dead: 0

Day 3
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 8
	Symptomatic: 4
	Recovered: 13
	Dead: 0

Day 4
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 4
	Symptomatic: 2
	Recovered: 19
	Dead: 0

Day 5
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 2
	Recovered: 20
	Dead: 0

Day 6
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 1
	Recovered: 21
	Dead: 0

Day 7
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 1
	Recovered: 21
	Dead: 0

Day 8
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 1
	Asymptomatic: 0
	Sym

In [12]:
to_return = OpenCLRunner.run_aspics_multi( 5, ITERATIONS, STUDY_AREA,params_manual,False,False,USE_GPU,False,10,calibration_params,disease_params,STORE_DETAILED_COUNTS,False)

Running models:   0%|          | 0/5 [00:00<?, ?it/s]/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


Running a simulation  {0}  based on the study area
Loading snapshot from data/snapshots/Rutland/cache.npz
Snapshot is 7 MB



Running models:  20%|██        | 1/5 [00:00<00:01,  2.61it/s]/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "



Day 0
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 14
	Symptomatic: 8
	Recovered: 3
	Dead: 0

Day 1
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 14
	Symptomatic: 7
	Recovered: 4
	Dead: 0

Day 2
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 12
	Symptomatic: 7
	Recovered: 6
	Dead: 0

Day 3
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 7
	Symptomatic: 7
	Recovered: 11
	Dead: 0

Day 4
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 7
	Symptomatic: 5
	Recovered: 13
	Dead: 0

Day 5
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 7
	Symptomatic: 4
	Recovered: 14
	Dead: 0

Day 6
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 5
	Symptomatic: 2
	Recovered: 18
	Dead: 0

Day 7
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 4
	Symptomatic: 1
	Recovered: 20
	Dead: 0

Day 8
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 3
	Symp


Running models:  40%|████      | 2/5 [00:00<00:01,  2.68it/s]/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


...


Day 0
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 11
	Symptomatic: 10
	Recovered: 4
	Dead: 0

Day 1
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 9
	Symptomatic: 8
	Recovered: 8
	Dead: 0

Day 2
	Susceptible: 33419
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 8
	Symptomatic: 6
	Recovered: 11
	Dead: 1

Day 3
	Susceptible: 33419
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 6
	Symptomatic: 4
	Recovered: 15
	Dead: 1

Day 4
	Susceptible: 33419
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 5
	Symptomatic: 4
	Recovered: 16
	Dead: 1

Day 5
	Susceptible: 33419
	Exposed: 0
	Presymptomatic: 1
	Asymptomatic: 4
	Symptomatic: 2
	Recovered: 19
	Dead: 1

Day 6
	Susceptible: 33419
	Exposed: 0
	Presymptomatic: 1
	Asymptomatic: 4
	Symptomatic: 2
	Recovered: 19
	Dead: 1

Day 7
	Susceptible: 33419
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 3
	Recovered: 20
	Dead: 1

Day 8
	Susceptible: 33419
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 2
	Symp


Running models:  80%|████████  | 4/5 [00:01<00:00,  2.83it/s]/Users/fbenitez/opt/anaconda3/envs/aspics/lib/python3.8/site-packages/pyopencl/__init__.py:266: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "



Day 0
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 10
	Symptomatic: 13
	Recovered: 2
	Dead: 0

Day 1
	Susceptible: 33420
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 9
	Symptomatic: 11
	Recovered: 5
	Dead: 0

Day 2
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 7
	Symptomatic: 10
	Recovered: 9
	Dead: 0

Day 3
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 6
	Symptomatic: 8
	Recovered: 12
	Dead: 0

Day 4
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 3
	Recovered: 20
	Dead: 0

Day 5
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 3
	Symptomatic: 2
	Recovered: 21
	Dead: 0

Day 6
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 2
	Symptomatic: 2
	Recovered: 22
	Dead: 0

Day 7
	Susceptible: 33420
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 2
	Symptomatic: 1
	Recovered: 23
	Dead: 0

Day 8
	Susceptible: 33419
	Exposed: 1
	Presymptomatic: 0
	Asymptomatic: 0
	Sym


Running models: 100%|██████████| 5/5 [00:01<00:00,  2.77it/s]


Day 0
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 13
	Symptomatic: 8
	Recovered: 4
	Dead: 0

Day 1
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 11
	Symptomatic: 6
	Recovered: 8
	Dead: 0

Day 2
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 8
	Symptomatic: 4
	Recovered: 13
	Dead: 0

Day 3
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 4
	Symptomatic: 2
	Recovered: 19
	Dead: 0

Day 4
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 1
	Symptomatic: 2
	Recovered: 22
	Dead: 0

Day 5
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 1
	Symptomatic: 2
	Recovered: 22
	Dead: 0

Day 6
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 1
	Symptomatic: 1
	Recovered: 23
	Dead: 0

Day 7
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 0
	Symptomatic: 0
	Recovered: 25
	Dead: 0

Day 8
	Susceptible: 33421
	Exposed: 0
	Presymptomatic: 0
	Asymptomatic: 0
	Symp

In [ ]:
params = OpenCLRunner.create_params_manually( parameters_file=PARAMETERS_FILE,study_area=STUDY_AREA)

In [ ]:
params

In [ ]:
simulator, snapshot, study_area, iterations = setup_sim(params)

In [ ]:
PARAMS = OpenCLRunner.create_params_manually(parameters_file=PARAMETERS_FILE)

In [ ]:
aspics_path  = os.path.join("../data", "snapshots","Rutland","cache.npz")
assert os.path.isfile(SNAPSHOT_FILEPATH), f"Snapshot doesn't exist: {SNAPSHOT_FILEPATH}"

In [ ]:
assert ITERATIONS < len(OBSERVATIONS), \
    f"Have more iterations ({ITERATIONS}) than observations ({len(OBSERVATIONS)})."

In [ ]:
OpenCLRunner.init(
    iterations = ITERATIONS,
    repetitions = REPETITIONS,
    study_area= STUDY_AREA,
    observations = OBSERVATIONS,
    use_gpu = USE_GPU,
    store_detailed_counts = STORE_DETAILED_COUNTS,
    parameters_file = PARAMETERS_FILE,
    snapshot_filepath = SNAPSHOT_FILEPATH,
    use_healthier_pop = False
)

In [ ]:
OpenCLRunner.update(repetitions=10)  # Temporarily use more repetitions to give a good baseline, initially suggested as 10
OpenCLRunner.update(study_area="Liverpool")
OpenCLRunner.update(store_detailed_counts=True)  # Temporarily output age breakdowns

Here I just try to read the model from ASPCIS

In [ ]:
PARAMETERS_FILE = 'config/Rutland.yml'

In [ ]:
with open(PARAMETERS_FILE, "r") as f:
    parameters = load(f, Loader=SafeLoader)

In [ ]:
(fitness0, sim0, obs0, out_params0, summaries0) = OpenCLRunner.run_model_with_params_abc({}, return_full_details=True)

In [ ]:
OpenCLRunner.update(repetitions=10)  # Temporarily use more repetitions to give a good baseline, initially suggested as 10

OpenCLRunner.update(store_detailed_counts=True)  # Temporarily output age breakdowns

(fitness0, sim0, obs0, out_params0, summaries0) = OpenCLRunner.run_model_with_params_abc({}, return_full_details=True)

OpenCLRunner.update(repetitions=REPETITIONS)

OpenCLRunner.update(store_detailed_counts=STORE_DETAILED_COUNTS)

# Check the model returns the observations correctly
np.array_equal(obs0, OBSERVATIONS.loc[:len(sim0)-1,"Cases"])

# Print the fitness and plot the different disease counts
print(f"fitness: {fitness0}")
#print(pd.DataFrame({"sim":sim, "real_obs1":obs}))

fig, ax = plt.subplots(1,1)
x = range(len(sim0))
ax.plot(x, OpenCLRunner.get_cumulative_new_infections(summaries0), label="sim", color="orange")
ax.plot(x, obs0, label="obs", color="blue")
ax.legend()